In [2]:
import pandas as pd
import sklearn
import time
from xgboost import XGBClassifier
from numpy import reshape
from sklearn import metrics

In [3]:
data = pd.read_csv('train_dec04_task2.csv')

In [4]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(data, test_size=0.2, random_state=0)

In [5]:
X_train = train_data.iloc[:, :7]
y_train = train_data.iloc[:, 7]

X_test = test_data.iloc[:, :7]
y_test = test_data.iloc[:, 7]

pred_data = pd.read_csv('test_dec04_task2_only_features.csv')

In [7]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.fit_transform(y_test)

In [8]:
# build XGBClassifier model
xgboostModel = XGBClassifier(n_estimators=100, learning_rate= 0.3)
# train data with train_data
xgboostModel.fit(X_train, y_train)
# predict with the test_data
predicted = xgboostModel.predict(X_train)

In [9]:
# 預測成功的比例
print('訓練集: ',xgboostModel.score(X_train,y_train))
print('測試集: ',xgboostModel.score(X_test,y_test))

訓練集:  1.0
測試集:  0.8122222222222222


In [10]:
# predict the real given data
answer = xgboostModel.predict(pred_data)
answer = le.inverse_transform(answer)

In [11]:
da = {"Id": [x for x in range(1, len(answer)+1)], "Category": answer}
df = pd.DataFrame (da)

In [12]:
df.to_csv("output.csv", index=False)